## 导包

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #默认为'last'
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import sympy as sy
import scipy as sc

# 这两个要单独导包
import scipy.linalg
import scipy.sparse.linalg

import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import sys
import random
from numpy import linalg
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)

# 行列式

# 矩阵及其运算

## 线性方程组和矩阵

### 对称矩阵

#### 创建对称矩阵

In [3]:
X = np.random.rand(5**2).reshape(5, 5)
X = np.triu(X) # 上三角矩阵
X
X.diagonal() # 对角元素
np.diag(X.diagonal()) # 对角矩阵
X += X.T - np.diag(X.diagonal())
X

array([[0.43, 0.86, 0.82, 0.2 , 0.5 ],
       [0.  , 0.89, 0.83, 0.12, 0.77],
       [0.  , 0.  , 0.5 , 0.43, 0.58],
       [0.  , 0.  , 0.  , 0.08, 0.92],
       [0.  , 0.  , 0.  , 0.  , 0.74]])

array([0.43, 0.89, 0.5 , 0.08, 0.74])

array([[0.43, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.89, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.5 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.08, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.74]])

array([[0.43, 0.86, 0.82, 0.2 , 0.5 ],
       [0.86, 0.89, 0.83, 0.12, 0.77],
       [0.82, 0.83, 0.5 , 0.43, 0.58],
       [0.2 , 0.12, 0.43, 0.08, 0.92],
       [0.5 , 0.77, 0.58, 0.92, 0.74]])

In [9]:
def gen_sym_matrix(n, type='randn', low=0, high=10, seed=None):
    '''
    生成实对称矩阵
    :param n: 生成对称矩阵的维度
    :param type: 生成随机数类型，默认标准正态分布
    :param low: 整数随机数上限，默认包含0
    :param high: 整数随机数上限，默认不包含10
    :param seed: 随机数种子，默认为None
    :return:
    '''
    np.random.seed(seed)
    if type == 'randint':
        X = np.random.randint(low, high, size=(n, n))
    elif type == 'rand':
        X = np.random.rand(n, n)
    else:
        X = np.random.randn(n, n)
    X = np.triu(X)  # 上三角矩阵
    X += X.T - np.diag(X.diagonal())
    return X


#### 特征值问题
* 标准特征值问题$Ax=\lambda x$
* 广义特征值问题$Ax=\lambda B x$


#### 特征值求解器
* LAPACK
    * 使用scipy.linalg模块的eigh函数及julia的eig函数（稠密矩阵）
* ARPACK
    * 使用scipy.sparse.linalg模块的eigsh函数及julia的eigs函数（稀疏矩阵）

#### 几个特征值求解函数的性能比较  (没调好)
[参考](https://fanyublog.com/2016/06/25/eig_rosser_python/)

In [23]:
# Rosser矩阵
Rosser = sy.Matrix([[611,196,-192,407,-8,-52,-49,29],
                 [196,899,113,-192,-71,-43,-8,-44],
                 [-192,113,899,196,61,49,8,52],
                 [407,-192,196,611,8,44,59,-23],
                 [-8,-71,61,8,411,-599,208,208],
                 [-52,-43,49,44,-599,411,208,208],
                 [-49,-8,8,59,208,208,99,-911],
                 [29,-44,52,-23,208,208,-911,99]])
Rosser

Matrix([
[ 611,  196, -192,  407,   -8,  -52,  -49,   29],
[ 196,  899,  113, -192,  -71,  -43,   -8,  -44],
[-192,  113,  899,  196,   61,   49,    8,   52],
[ 407, -192,  196,  611,    8,   44,   59,  -23],
[  -8,  -71,   61,    8,  411, -599,  208,  208],
[ -52,  -43,   49,   44, -599,  411,  208,  208],
[ -49,   -8,    8,   59,  208,  208,   99, -911],
[  29,  -44,   52,  -23,  208,  208, -911,   99]])

In [58]:
# m = gen_sym_matrix(3, type='randint',low=0,high=3,seed=0)
m = Rosser
# m
R = sy.Matrix(m)
# R
re = R.eigenvals()
# re
for key in re.keys():
    print('%30s,%5s,\t%f'%(key,re[key],key.evalf()))

                          1020,    1,	1020.000000
                          1000,    2,	1000.000000
            510 - 100*sqrt(26),    1,	0.098049
            100*sqrt(26) + 510,    1,	1019.901951
               -10*sqrt(10405),    1,	-1020.049018
                10*sqrt(10405),    1,	1020.049018
                             0,    1,	0.000000


In [69]:
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
import scipy.sparse
# from scipy.sparse.linalg import eigsh
# from scipy.linalg import eigh
# np.linalg.eig(m)
N=100
m1 = scipy.sparse.rand(N, N, density=0.1)
eigsh(m1)

(array([-3.45, -3.4 , -3.3 ,  3.31,  3.36,  5.72]),
 array([[ 0.1 ,  0.09, -0.04, -0.02, -0.03,  0.12],
        [-0.1 , -0.03,  0.05, -0.22,  0.04,  0.07],
        [-0.01,  0.05, -0.06,  0.05, -0.23,  0.1 ],
        [-0.02,  0.07,  0.04, -0.03, -0.01,  0.06],
        [ 0.1 ,  0.  ,  0.05,  0.08,  0.12,  0.08],
        [ 0.09, -0.05, -0.03,  0.03,  0.01,  0.07],
        [-0.05,  0.18,  0.01, -0.02,  0.01,  0.09],
        [-0.03,  0.02, -0.03, -0.18, -0.05,  0.1 ],
        [-0.08, -0.01, -0.17,  0.13,  0.06,  0.19],
        [-0.01,  0.14, -0.03,  0.03,  0.21,  0.12],
        [-0.04, -0.21, -0.11,  0.06, -0.01,  0.1 ],
        [ 0.03,  0.13, -0.11,  0.08, -0.22,  0.11],
        [-0.09, -0.1 , -0.05,  0.05,  0.02,  0.1 ],
        [ 0.09,  0.02,  0.08, -0.04,  0.16,  0.09],
        [ 0.06, -0.02,  0.07, -0.1 , -0.06,  0.1 ],
        [-0.  , -0.  , -0.01,  0.28,  0.12,  0.11],
        [ 0.11,  0.06,  0.02,  0.06,  0.02,  0.16],
        [ 0.06,  0.01,  0.01, -0.06,  0.03,  0.05],
        [-0.

In [4]:
import scipy.linalg
import scipy.sparse.linalg
funcdic = {0:('numpy.linalg.eig',np.linalg.eig,{}),
           1:('numpy.linalg.eigh',np.linalg.eigh,{}),
           2:('scipy.linalg.eig',sc.linalg.eig,{}),
           3:('scipy.linalg.eigh',sc.linalg.eigh,{}),
           4:('scipy.sparse.linalg.eigs',sc.sparse.linalg.eigs,{}),
           5:('scipy.sparse.linalg.eigsh',sc.sparse.linalg.eigsh,{}),
           6:('scipy.sparse.linalg.eigs, sigma=0',sc.sparse.linalg.eigs,{'sigma':0}),
           7:('scipy.sparse.linalg.eigsh, sigma=0',sc.sparse.linalg.eigsh,{'sigma':0})}
 

In [56]:
def timing_val(func):
    def wrapper(*arg, **kw):
        t1 = time.time()
        for i in range(0,1000):
            res = func(*arg, **kw)
        t2 = time.time()
        print('%40s  %.3e sec' % (funcdic[arg[0]][0], t2-t1))
        return [res,t2-t1]
    return wrapper
@timing_val
def test_func(i):
    name,func,args = funcdic[i]
    eigval,eigvec = func(m,**args)
    return numpy.real(numpy.sort(eigval))
 
def compare():
    Resultslist = []
    for i in range(0,8):
        try:
            Resultslist.append(test_func(i))
        except Exception as e:
            print(type(e))
            print(e)
        continue
    return Resultslist

In [57]:
Resultslist = compare()
Resultslist
# exact = [-1020.049018,0,0.098049,1000,1000,1019.901951,1020.000000,1020.049018]

# for i in range(0,8):
#     name,func,args = funcdic[i]
#     print('='*20)
#     print(name) 
#     for j in range(0,8):
#         if j < len(Resultslist[i][0]):
#             print('%20f,%20f'%(exact[j],Resultslist[i][0][j])) 
#         else:
#             print('%20f,'%(exact[j],)) 
#     print('='*20) 
#     print('\n\n') 

# info = 'method index - details\n'
# for i in range(0,8):
#     info = info + '%d - %s\n'%(i,funcdic[i][0])

# plt.figure()
# tt = [Resultslist[i][1] for i in range(0,8)]
# plt.bar(numpy.arange(0,8)-0.4, tt, width=0.8,alpha=0.6)
# plt.xticks(range(0,8),[str(i) for i in range(0,8)])
# plt.xlabel('method index',fontsize = 16)
# plt.ylabel('time (s)',fontsize = 16)
# plt.text(0,0.5,info,fontsize = 14)
# plt.show()

<class 'TypeError'>
ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
<class 'TypeError'>
No loop matching the specified signature and casting was found for ufunc eigh_lo
<class 'ValueError'>
object arrays are not supported
<class 'ValueError'>
object arrays are not supported
<class 'TypeError'>
type not understood
<class 'AttributeError'>
MutableDenseMatrix has no attribute dtype.
<class 'AttributeError'>
MutableDenseMatrix has no attribute dtype.
<class 'AttributeError'>
MutableDenseMatrix has no attribute dtype.


[]

In [19]:
M = sy.Matrix([[3, -2,  4, -2], [5,  3, -3, -2], [5, -2,  2, -2], [5, -2, -3,  3]])
M.eigenvals()
M.eigenvects()

{3: 1, -2: 1, 5: 2}

[(-2,
  1,
  [Matrix([
   [0],
   [1],
   [1],
   [1]])]),
 (3,
  1,
  [Matrix([
   [1],
   [1],
   [1],
   [1]])]),
 (5,
  2,
  [Matrix([
   [1],
   [1],
   [1],
   [0]]),
   Matrix([
   [ 0],
   [-1],
   [ 0],
   [ 1]])])]

## 矩阵的运算

## 逆矩阵

## 克拉默法则

## 矩阵分块法

# 矩阵的初等变换与线性方程组

# 向量组的线性相关性

# 相似矩阵及二次型

## 向量的内积、长度及正交性

## 方阵的特征值与特征向量

## 相似矩阵

## 对称矩阵的对角化

## 二次型及其标准型

## 用配方法化二次型为标准型

## 正定二次型

# 线性空间与线性变换

# 其它

## 矩阵LU分解
[Python实例：矩阵LU分解](https://blog.csdn.net/weixin_34367257/article/details/91716112)

In [115]:
import scipy.linalg as la
import numpy.linalg as LA
from scipy import optimize
'''
求解线性方程组
2x1 + 3x2 = 4
5x1 + 4x2 = 3
'''
 
# 使用符号方式求解
A1 = sy.Matrix([[2,3],[5,4]])
b1 = sy.Matrix([4,3])
A1
print('rank:',A1.rank()) # 秩
print('condition number:',A1.condition_number()) # 条件数？
# an1 =  A1.norm()
# an2 = A1.inv().norm()
# sy.simplify(an1)
# sy.simplify(an2)
# ac1 = A1.condition_number()
# ac2 = an1*an2
# sy.simplify(ac1)
# sy.simplify(ac2)
print('A.norm():',A1.norm(),A1.norm().evalf()) # 2范数：最大奇异值的平方根
"***"
# LU分解
L1, U1, _ = A1.LUdecomposition()
print('L,U = ',L1,U1)
print('L * U = ',(L1 * U1)) # equivalent to A.LUsolve(b)
x1 = A1.solve(b1)
print('x = ',x1)
"***"
# 使用scipy线性库求解
A2 = np.array([[2, 3], [5, 4]])
b2 = np.array([4, 3])
print('rank:',np.linalg.matrix_rank(A2))
print('cond:',np.linalg.cond(A2))
"***" 
# LU分解
P2, L2, U2 = la.lu(A2)
print('L,U = ',L2,U2)
print('L @ U = ',L2 @ U2)
x2 = la.solve(A2,b2)
print('x = ',x2)

# # LU分解2
# P3, L3, U3 = LA.lu(A2)
# print('L,U = ',L3,U3)
# print('L @ U = ',L3 @ U3)
# # x3 = la.solve(A2,b2)
# # print('x3 = ',x3)


'\n求解线性方程组\n2x1 + 3x2 = 4\n5x1 + 4x2 = 3\n'

Matrix([
[2, 3],
[5, 4]])

rank: 2
condition number: sqrt(2*sqrt(170) + 27)/sqrt(27 - 2*sqrt(170))
A.norm(): 3*sqrt(6) 7.34846922834953


'***'

L,U =  Matrix([[1, 0], [5/2, 1]]) Matrix([[2, 3], [0, -7/2]])
L * U =  Matrix([[2, 3], [5, 4]])
x =  Matrix([[-1], [2]])


'***'

rank: 2
cond: 7.582401374401514


'***'

L,U =  [[1.  0. ]
 [0.4 1. ]] [[5.  4. ]
 [0.  1.4]]
L @ U =  [[5. 4.]
 [2. 3.]]
x =  [-1.  2.]


In [107]:
# 说明这里的条件数为奇异值中最大值与最小值的比值
_,S1,_ = linalg.svd(A2)
S1
S1[0]/S1[1]
A1.condition_number().evalf()

array([7.29, 0.96])

7.582401374401514

7.58240137440151

In [114]:
# help(np.linalg.cond)
dir(LA)

['LinAlgError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_umath_linalg',
 'absolute_import',
 'cholesky',
 'cond',
 'det',
 'division',
 'eig',
 'eigh',
 'eigvals',
 'eigvalsh',
 'inv',
 'lapack_lite',
 'linalg',
 'lstsq',
 'matrix_power',
 'matrix_rank',
 'multi_dot',
 'norm',
 'pinv',
 'print_function',
 'qr',
 'slogdet',
 'solve',
 'svd',
 'tensorinv',
 'tensorsolve',
 'test']

In [91]:
A1 = sy.Matrix([[2,3],[5,4]])
help(A.norm)
# help(np.linalg.norm)

Help on method norm in module sympy.matrices.matrices:

norm(ord=None) method of sympy.matrices.dense.MutableDenseMatrix instance
    Return the Norm of a Matrix or Vector.
    In the simplest case this is the geometric size of the vector
    Other norms can be specified by the ord parameter
    
    
    =====  ============================  ==========================
    ord    norm for matrices             norm for vectors
    =====  ============================  ==========================
    None   Frobenius norm                2-norm
    'fro'  Frobenius norm                - does not exist
    inf    maximum row sum               max(abs(x))
    -inf   --                            min(abs(x))
    1      maximum column sum            as below
    -1     --                            as below
    2      2-norm (largest sing. value)  as below
    -2     smallest singular value       as below
    other  - does not exist              sum(abs(x)**ord)**(1./ord)
    =====  ============

In [93]:
A1 = sy.Matrix([[2,3],[5,4]])
help(A.condition_number)

Help on method condition_number in module sympy.matrices.matrices:

condition_number() method of sympy.matrices.dense.MutableDenseMatrix instance
    Returns the condition number of a matrix.
    
    This is the maximum singular value divided by the minimum singular value
    
    Examples
    
    >>> from sympy import Matrix, S
    >>> A = Matrix([[1, 0, 0], [0, 10, 0], [0, 0, S.One/10]])
    >>> A.condition_number()
    100
    
    See Also
    
    singular_values



In [97]:
from sympy import Matrix, S
A1 = Matrix([[1, 1, 0], [0, 10, 0], [0, 0, 2/10]])
A1
A1.condition_number()

Matrix([
[1,  1,   0],
[0, 10,   0],
[0,  0, 0.2]])

5*sqrt(sqrt(2501) + 51)